In [ ]:
import os
import cv2
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

# Uneven illumination generator
# Just add any RGB image to the content folder (One or multiple)

def resize_images(images, target_size=(128, 128)):
    return np.array([cv2.resize(img, target_size) for img in images])

def uneven_illumination_rgb(image, max_illumination=1.0, min_illumination=0.5, smoothness=75):
    image = image.astype(float)
    rows, cols = image.shape[:2]
    gradient_type = np.random.choice(['linear', 'circular', 'diagonal'])

    mask = np.zeros((rows, cols))

    if gradient_type == 'linear':
        start, end = np.random.randint(0, cols, 2)
        start, end = min(start, end), max(start, end)
        mask[:, start:end] = np.linspace(min_illumination, max_illumination, end - start).reshape(1, -1)
    elif gradient_type == 'circular':
        center = [np.random.randint(low=0, high=rows), np.random.randint(low=0, high=cols)]
        max_distance_to_center = max(np.sqrt((rows - center[0])**2 + (cols - center[1])**2),
                                     np.sqrt(center[0]**2 + center[1]**2))
        for i in range(rows):
            for j in range(cols):
                distance_to_center = np.sqrt((i - center[0])**2 + (j - center[1])**2)
                mask[i, j] = min_illumination + (max_illumination - min_illumination) * (distance_to_center / max_distance_to_center)
    elif gradient_type == 'diagonal':
        diagonal_start = np.random.choice(['top_left', 'top_right', 'bottom_left', 'bottom_right'])
        for i in range(rows):
            for j in range(cols):
                if diagonal_start == 'top_left':
                    mask[i, j] = min_illumination + (max_illumination - min_illumination) * ((i + j) / (rows + cols - 2))
                elif diagonal_start == 'top_right':
                    mask[i, j] = min_illumination + (max_illumination - min_illumination) * ((i + (cols - j)) / (rows + cols - 2))
                elif diagonal_start == 'bottom_left':
                    mask[i, j] = min_illumination + (max_illumination - min_illumination) * (((rows - i) + j) / (rows + cols - 2))
                elif diagonal_start == 'bottom_right':
                    mask[i, j] = min_illumination + (max_illumination - min_illumination) * (((rows - i) + (cols - j)) / (rows + cols - 2))

    mask = cv2.GaussianBlur(mask, (smoothness, smoothness), 0)

    image_masked = np.zeros_like(image)
    for c in range(3):
        image_masked[:, :, c] = cv2.multiply(image[:, :, c], mask)

    return image_masked, mask



def load_images_from_folder(folder):
    images = []
    for filename in os.listdir(folder):
        img = cv2.imread(os.path.join(folder, filename), cv2.IMREAD_UNCHANGED)
        if img is not None:
            img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
            img = img.astype('float32') / 255
            images.append(img)
    return np.array(images)

def resize_images(images, target_size=(128, 128)):
    return np.array([cv2.resize(img, target_size, interpolation=cv2.INTER_AREA) for img in images])

def generate_noisy_images(images, num_samples=1500):
    noisy_images = []
    illumination_masks = []
    for image in images:
        for _ in range(num_samples):
            max_illumination = np.random.uniform(0.7, 1.0)
            min_illumination = np.random.uniform(0.1, max_illumination)
            noisy_image, illumination_mask = uneven_illumination_rgb(image, max_illumination, min_illumination)
            noisy_images.append(noisy_image)
            illumination_masks.append(illumination_mask)
    return np.array(noisy_images), np.array(illumination_masks)

folder_path = '/content'
x_train = load_images_from_folder(folder_path)
x_train = resize_images(x_train)

x_train_noisy, x_train_masks = generate_noisy_images(x_train,num_samples=1500)
num_samples_per_image = len(x_train_noisy) // len(x_train)
x_train_repeated = np.repeat(x_train, num_samples_per_image, axis=0)

x_train, x_test, x_train_noisy, x_test_noisy, x_train_masks, x_test_masks = train_test_split(
 x_train_repeated, x_train_noisy, x_train_masks, test_size=0.1, random_state=42)
plt.figure(figsize=(15, 5))
plt.subplot(1, 3, 1)
plt.imshow(x_train[0])
plt.title("Original Image")
plt.subplot(1, 3, 2)
plt.imshow(x_train_noisy[0])
plt.title("Noisy Image")
plt.subplot(1, 3, 3)
plt.imshow(x_train_masks[0], cmap='gray')
plt.title("Illumination Mask")
plt.show()

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
#plotting 10 examples
n = min(10, len(x_train_noisy), len(x_train_masks))

plt.figure(figsize=(15, n * 4))
for idx in range(n):
    ax = plt.subplot(n, 3, idx * 3 + 1)
    plt.imshow(x_train[idx])
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)

    ax = plt.subplot(n, 3, idx * 3 + 2)
    plt.imshow(x_train_noisy[idx])
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)

    ax = plt.subplot(n, 3, idx * 3 + 3)
    plt.imshow(x_train_masks[idx], cmap='gray')
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)

plt.tight_layout()
plt.show()

In [ ]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, UpSampling2D, concatenate, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
from tensorflow.keras.regularizers import l2
import tensorflow as tf


def unet_model(input_size=(128, 128, 3)):
    inputs = Input(input_size)

    conv1 = Conv2D(64, 3, activation='relu', padding='same', kernel_initializer='he_normal', kernel_regularizer=l2(0.001))(inputs)
    pool1 = MaxPooling2D(pool_size=(2, 2))(conv1)

    conv2 = Conv2D(128, 3, activation='relu', padding='same', kernel_initializer='he_normal', kernel_regularizer=l2(0.001))(pool1)
    pool2 = MaxPooling2D(pool_size=(2, 2))(conv2)

    conv3 = Conv2D(256, 3, activation='relu', padding='same', kernel_initializer='he_normal', kernel_regularizer=l2(0.001))(pool2)
    pool3 = MaxPooling2D(pool_size=(2, 2))(conv3)

    conv4 = Conv2D(512, 3, activation='relu', padding='same', kernel_initializer='he_normal', kernel_regularizer=l2(0.001))(pool3)

    up1 = Conv2D(512, 2, activation='relu', padding='same', kernel_initializer='he_normal')(UpSampling2D(size=(2, 2))(conv4))
    merge1 = concatenate([conv3, up1], axis=3)
    conv5 = Conv2D(256, 3, activation='relu', padding='same', kernel_initializer='he_normal')(merge1)

    up2 = Conv2D(256, 2, activation='relu', padding='same', kernel_initializer='he_normal')(UpSampling2D(size=(2, 2))(conv5))
    merge2 = concatenate([conv2, up2], axis=3)
    conv6 = Conv2D(128, 3, activation='relu', padding='same', kernel_initializer='he_normal')(merge2)

    up3 = Conv2D(128, 2, activation='relu', padding='same', kernel_initializer='he_normal')(UpSampling2D(size=(2, 2))(conv6))
    merge3 = concatenate([conv1, up3], axis=3)
    conv7 = Conv2D(64, 3, activation='relu', padding='same', kernel_initializer='he_normal')(merge3)

    conv8 = Conv2D(1, 1)(conv7)

    model = Model(inputs=inputs, outputs=conv8)
    return model


model = unet_model()
ModelCheckpoint.compile(optimizer = tf.keras.optimizers.Adam(learning_rate=0.001), loss='MeanSquaredError', metrics=['MAE'])

early_stopping = EarlyStopping(patience=15, verbose=1)

model_checkpoint = ModelCheckpoint('unet_FLirduo_proR_RGB.hdf5', monitor='loss', verbose=1, save_best_only=True)
model.fit(x_train_noisy , x_train_masks,
          epochs=25,
          batch_size=32,
          shuffle=True,
          callbacks=[model_checkpoint, early_stopping],
          validation_data=(x_test_noisy, x_test_masks))

In [ ]:
mport os
import cv2
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, UpSampling2D, concatenate, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
from tensorflow.keras.regularizers import l2
import tensorflow as tf
import numpy as np
def process_and_correct_images(folder_path):
    corrected_folder_path = os.path.join(folder_path, 'corrected')

    if not os.path.exists(corrected_folder_path):
        os.makedirs(corrected_folder_path)
        print(f"Created directory: {corrected_folder_path}")

    for filename in os.listdir(folder_path):
        if filename.lower().endswith('.jpg'):
            img_path = os.path.join(folder_path, filename)

            img = cv2.imread(img_path, cv2.IMREAD_UNCHANGED)
            if img is None:
                print(f"Failed to load image: {img_path}")
                continue

            img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

            original_size = img.shape[:2]

            img_resized = cv2.resize(img, (128, 128))
            img_resized = img_resized[np.newaxis, ...] / 255.0

            predicted_mask = model_RGB.predict(img_resized)
            predicted_mask = np.squeeze(predicted_mask)

            predicted_mask_resized = cv2.resize(predicted_mask, (original_size[1], original_size[0]))

            corrected_image = np.zeros_like(img, dtype=np.float32)
            for c in range(3):
                corrected_image[..., c] = img[..., c] / predicted_mask_resized


            new_filename = filename[:-4] + '_corrected.jpg'
            save_path = os.path.join(corrected_folder_path, new_filename)

            cv2.imwrite(save_path, cv2.cvtColor(corrected_image, cv2.COLOR_RGB2BGR))
            print(f"Corrected image saved to: {save_path}")

# Specify the directory containing the JPEG images to be corrected
folder_path = "/content"
process_and_correct_images(folder_path)

